# List of Exposures  at USDF on Photometry


- work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/30
- last update : 2024/02/26
  


In [ ]:
import sys
print(sys.executable)

In [ ]:
print(sys.path)

In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [ ]:
from IPython.display import display, HTML

In [ ]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
#my_collection="LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

In [ ]:
#file_out = "AUXTEL_DRP_IMAGING_2023-ListOfExposures.csv"
file_out = "AUXTEL_DRP_IMAGING_20230509_20240201-ListOfExposures.csv"

## List of Exposures

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [ ]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    try:
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

In [ ]:
info

In [ ]:
df_exposure

In [ ]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [ ]:
df_exposure

In [ ]:
df_exposure.type.unique()

In [ ]:
df_exposure.science_program.unique()

In [ ]:
#SCIENCE_PROGRAM = 'AUXTEL_DRP_IMAGING'
SCIENCE_PROGRAM = ['AUXTEL_DRP_IMAGING','AUXTEL_PHOTO_IMAGING']

## Selection of science

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
df_science.head(60)

In [ ]:
df_science.tail(60)

In [ ]:
df_science.day_obs.unique()

## Query Dimension records on Exposures

In [ ]:
#where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'AUXTEL_DRP_IMAGING\'"
where_clause = f"instrument=\'LATISS\' AND exposure.science_program=\'AUXTEL_DRP_IMAGING\'"

In [ ]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = my_collection,where=where_clause)
#results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = my_collection)

In [ ]:
results_exposures 

## Select date

In [ ]:
cut_science_program = (df_science["science_program"] == SCIENCE_PROGRAM[0]) |  (df_science["science_program"] == SCIENCE_PROGRAM[1])

In [ ]:
df_science_selected = df_science[cut_science_program]

In [ ]:
len(df_science_selected)

In [ ]:
df_science.day_obs.unique()

In [ ]:
#df_science_selected

# List of filters

In [ ]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

In [ ]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('empty') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

In [ ]:
df_science_selected.to_csv(file_out)